In [20]:
import os
import cdsapi

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as p9
import seaborn as sns

In [21]:
#!pip install sdcpy
from tqdm.auto import tqdm
from collections import defaultdict
from sdcpy.scale_dependent_correlation import SDCAnalysis
import warnings

# Cross-correlation between COVID and climate

In [59]:
world = pd.read_pickle('../data/world_shape.pkl')

In [60]:
country_to_iso = world[['country', 'iso3']].drop_duplicates()

In [42]:
covid_df = pd.read_csv('../data/owd/covid19_world.csv')

In [43]:
dens = pd.read_csv('../data/processed_05_grid.csv')

In [44]:
country_groups = pd.read_csv('../data/country_groups.csv')

In [45]:
lon_lat_country = pd.read_csv('../data/coords_region.csv').merge(country_to_iso)

In [46]:
lon_lat_group = lon_lat_country.merge(country_groups)

In [47]:
processed_path = "../data/climate/processed/"
relevant_files = [f for f in os.listdir(processed_path) if f[:9] < '2021_07_0']
climate_df = pd.concat([pd.read_pickle(f'{processed_path}{f}') for f in relevant_files])

In [ ]:
f = (world
     .merge(country_groups)
     .pipe(lambda dd: p9.ggplot(dd)
       + p9.geom_map(data=world, fill='gray')
       + p9.geom_map(p9.aes(fill='group'))
       + p9.ylim(-55, None)
       + p9.labs(fill='')
       + p9.theme_void()
       + p9.theme(figure_size=(8, 4), dpi=200, legend_key_height=15)
      )
)
f

In [49]:
weighted_cells = (dens.merge(lon_lat_group)
    .groupby('group', as_index=False)
    .apply(lambda gdd: gdd.assign( relative_weight=lambda gdd: gdd.density / gdd.density.sum()))
    .reset_index(drop=True)
)

C:\Users\brand\AppData\Local\Temp\ipykernel_18208\1766496956.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [50]:
weighted_climate_df = (
        weighted_cells.dropna()
        .merge(climate_df, on=["latitude", "longitude"])
        .assign(temperature=lambda dd: dd.temperature * dd.relative_weight,
                absolute_humidity=lambda dd: dd.absolute_humidity * dd.relative_weight)
        .groupby(['group', "date"], as_index=False)
        [["temperature", "absolute_humidity"]]
        .agg({"temperature": "sum", "absolute_humidity": "sum"})
)

In [51]:
groups_covid_ts = \
(covid_df
 .merge(country_groups)
 .assign(date=lambda dd: pd.to_datetime(dd.date))
 .groupby('group')
 .apply(lambda dd: 
        dd.set_index('date')
        .resample('D')
        .sum()
        ['new_cases']
        #.rolling(center=True, window=7)
        #.mean()
        .loc['2020-11-01': '2021-06-23'])
)

C:\Users\brand\AppData\Local\Temp\ipykernel_18208\4116118787.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [ ]:
groups_covid_ts

In [53]:
groups_weather_ts = \
(weighted_climate_df
 .assign(date=lambda dd: pd.to_datetime(dd.date))
 .query('"2020-11-01" <= date <= "2021-06-23"')
 .set_index(['group', 'date'])
)

In [54]:
CV_LABELS = {'absolute_humidity': 'Absolute Humidity [g/m³]',
             'temperature': 'Temperature [°C]',
             'relative_humidity': 'Relative Humidity [%]',
             'total_precipitation': 'Total Precipitation [mm]'}

In [57]:
import openpyxl

w = 75
out_dir = '../results'
for group in tqdm(country_groups.group.unique(), desc='Processing Groups', leave=False):
    for weather_variable in ['temperature', 'absolute_humidity']:
        covid_ts = groups_covid_ts.loc[group]
        weather_ts = groups_weather_ts.loc[group][weather_variable]
        sdc = SDCAnalysis(ts1=weather_ts,
                          ts2=covid_ts,
                          fragment_size=w,
                          method='spearman',
                          max_lag=0,
                          min_lag=-21)
        sdc.to_excel(f"{out_dir}/tables/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.xlsx")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sdc.combi_plot(dpi=120,
                           figsize=(7, 7),
                           title=f"SDC with s={w}: \nCOVID19 daily confirmed cases in" +
                                 f" {group} ~ {weather_variable}",
                           xlabel=CV_LABELS[weather_variable],
                           ylabel=f"Confirmed COVID19 cases in {group}",
                           alpha=0.05,
                           max_lag=0,
                           min_lag=-21,
                           max_r=1,
                           wspace=.35,
                           hspace=.35)
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.png")
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.pdf")